In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-30T16:33:16.813922-07:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.13.2

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.15.0-1017-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

/home/dgxuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%watermark --iversions

optuna : 3.1.1
dask   : 2023.3.2
logging: 0.5.1.2
xgboost: 1.7.5
pandas : 1.5.3
numpy  : 1.23.5



In [5]:
!nvidia-smi -L

GPU 0: NVIDIA H100 80GB HBM3 (UUID: GPU-5c583ee7-8fb1-b26a-4bea-16e45d984a32)
GPU 1: NVIDIA H100 80GB HBM3 (UUID: GPU-a6ab06f5-a6e2-18c1-9dec-0dcd29f44a46)
GPU 2: NVIDIA H100 80GB HBM3 (UUID: GPU-bb8d5098-3c56-c48d-a0a3-fcdfcec6d3f5)
GPU 3: NVIDIA H100 80GB HBM3 (UUID: GPU-cdbe686b-1611-999b-8e8d-a3c5f35b40c4)
GPU 4: NVIDIA H100 80GB HBM3 (UUID: GPU-0df1cef0-fc95-cc88-b5f4-239889b3acba)
GPU 5: NVIDIA H100 80GB HBM3 (UUID: GPU-9dca657c-dbe4-08b7-fe7c-5653f183f0b6)
GPU 6: NVIDIA H100 80GB HBM3 (UUID: GPU-33389782-e2ad-5022-997d-cf470313879c)
GPU 7: NVIDIA H100 80GB HBM3 (UUID: GPU-926eaa05-87fb-35e0-0a98-adb2ad41d0be)


In [6]:
cluster = LocalCUDACluster(n_workers=8)
client = Client(cluster)

2023-05-30 16:33:28,216 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:33:28,216 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:33:28,216 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 16:33:28,216 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 16:33:28,216 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:33:28,216 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:33:28,216 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 16:33:28,216 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:33:28,216 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 16:33:28,216 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-

In [7]:
!nvidia-smi

Tue May 30 16:33:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA H100 80G...  On   | 00000000:1B:00.0 Off |                    0 |
| N/A   32C    P0   109W / 700W |    525MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA H100 80G...  On   | 00000000:43:00.0 Off |                    0 |
| N/A   

In [8]:
%%time
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input3/xgtrain_fold_{i}_l.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input3/xgval_fold_{i}_l.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4
CPU times: user 2.27 s, sys: 2.74 s, total: 5.01 s
Wall time: 1min 27s


In [9]:
train = pd.read_csv('../input/train.csv.zip')

shift = 200

target0 = train['loss'].values
target = np.log(target0+shift)

In [10]:
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con =2
    x =preds-labels
    grad =con*x / (np.abs(x)+con)
    hess =con**2 / (np.abs(x)+con)**2
    return grad, hess 


def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

In [11]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        #'objective': 'reg:squarederror', 
        'base_score':7.76,
        'tree_method':'gpu_hist',  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['rmse']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_boost_round=num_round, obj=logregobj)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = np.exp(predictions) - shift
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    mae = mean_absolute_error(target0, train_oof)
    
    return mae

In [12]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_l_7.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_allstate_l_7.db", study_name="five_fold_optuna_xgb_l_7", direction='minimize')

In [13]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/tmp/ipykernel_1748655/481481954.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_1748655/481481954.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_1748655/481481954.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_1748655/481481954.py:14: FutureWarning: s

CPU times: user 56.4 s, sys: 42 s, total: 1min 38s
Wall time: 18min 33s


In [14]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1275.218696,0.007709,0.816838,rmse,1.625922,0.003468,0.002068,6,232,0.499923,COMPLETE
1,1,1140.893318,7.468296,0.726356,rmse,0.209496,0.001005,0.028455,9,110,0.968466,COMPLETE
2,2,1149.789258,0.002079,0.543064,rmse,7.061484,0.237957,0.020644,16,71,0.657497,COMPLETE


In [15]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l_7.csv', index=False)
df

/tmp/ipykernel_1748655/481481954.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_1748655/481481954.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_1748655/481481954.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_1748655/481481954.py:14: FutureWarning: s

CPU times: user 1min 53s, sys: 2min 6s, total: 4min
Wall time: 51min 59s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1275.218696,0.007709,0.816838,rmse,1.625922,0.003468,0.002068,6,232,0.499923,COMPLETE
1,1,1140.893318,7.468296,0.726356,rmse,0.209496,0.001005,0.028455,9,110,0.968466,COMPLETE
2,2,1149.789258,0.002079,0.543064,rmse,7.061484,0.237957,0.020644,16,71,0.657497,COMPLETE
3,3,1197.336173,0.455604,0.388019,rmse,0.001672,0.008456,0.082131,16,156,0.823348,COMPLETE
4,4,1276.158187,0.006109,0.941649,rmse,0.070431,0.158820,0.001511,9,23,0.760617,COMPLETE
5,5,1159.488537,0.184908,0.685121,rmse,1.579037,1.742991,0.003539,16,160,0.572480,COMPLETE
6,6,1156.465877,0.128228,0.893958,rmse,0.490288,0.636472,0.039961,13,250,0.779027,COMPLETE
7,7,1253.955672,3.273318,0.444776,rmse,0.014412,3.388966,0.003717,4,10,0.530034,COMPLETE


In [ ]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l_7.csv', index=False)
df.head(30)

/tmp/ipykernel_1748655/481481954.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_1748655/481481954.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_1748655/481481954.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_1748655/481481954.py:14: FutureWarning: s

In [ ]:
df

In [ ]:
df.value.min()